# Data Understanding

## Preperation

Import packages and set globals

In [88]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.width", 1000)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 6)

Import the pattern.pkl file

In [89]:
df = pickle.load(open("../data/raw/naics_pattern.pkl", "rb"))

## Structure

Brief overview of the dataset

### Data Format

In [90]:
df.head(10)

State_GEOID County_GEOID  FIPS naics_2 naics                                        DESCRIPTION emp_nf  emp qp1_nf   qp1 ap_nf     ap  est n<5 n5_9 n10_19 n20_49 n50_99 n100_249 n250_499 n500_999 n1000 n1000_1 n1000_2 n1000_3 n1000_4
0           01          001  1001      11  1133                                            Logging      G   68      G  1213     G   4563    7   6    N      N      N      N        N        N        N     N       N       N       N       N
1           01          001  1001      21  2123           Nonmetallic Mineral Mining and Quarrying      G   87      G  1224     G   5144    6   N    N      N      N      N        N        N        N     N       N       N       N       N
2           01          001  1001      22  2211  Electric Power Generation, Transmission and Di...      G  129      G  4418     G  16342    4   N    N      N      N      N        N        N        N     N       N       N       N       N
4           01          001  1001      23  2362               Nonresidential Building Construction      H   69      H   685     H   4184    7   3    N      N      N      N        N        N        N     N       N       N       N       N
5           01          001  1001      23  2371                        Utility System Construction      H   65      H  1117     H   4574    4   N    N      N      N      N        N        N        N     N       N       N       N       N
6           01          001  1001      23  2381  Foundation, Structure, and Building Exterior C...      H   44      G   401     G   1775   12   7    4      N      N      N        N        N        N     N       N       N       N       N
7           01          001  1001      23  2382                     Building Equipment Contractors      H  157      G  1514     G   7822   27  15    7      4      N      N        N        N        N     N       N       N       N       N
8           01          001  1001      23  2383                     Building Finishing Contractors      H   81      H   753     H   3630   13   7    5      N      N      N        N        N        N     N       N       N       N       N
10          01          001  1001      32  3231            Printing and Related Support Activities      J   33      H   280     H   1334    4   N    N      N      N      N        N        N        N     N       N       N       N       N
11          01          001  1001      42  4238  Machinery, Equipment, and Supplies Merchant Wh...      J   51      J   867     H   3847    6   3    N      N      N      N        N        N        N     N       N       N       N       N

In [91]:
df.dtypes

State_GEOID     object
County_GEOID    object
FIPS             int64
naics_2         object
naics           object
DESCRIPTION     object
emp_nf          object
emp              int64
qp1_nf          object
qp1              int64
ap_nf           object
ap               int64
est              int64
n<5             object
n5_9            object
n10_19          object
n20_49          object
n50_99          object
n100_249        object
n250_499        object
n500_999        object
n1000           object
n1000_1         object
n1000_2         object
n1000_3         object
n1000_4         object
dtype: object

### Remove unnecessary data for our task

NAICS which don't fit our market:
- not in metal working
- focus on 11, 21, 22, 23, 31-33 

In [92]:
# Remove NAICS as stated
naics_filter = "|".join(["^11", "^21", "^22", "^23", "^31", "^32", "^33"])
df = df.loc[df["naics"].str.contains(naics_filter)]

### Content Description

In [93]:
df.describe()

FIPS      emp        qp1         ap      est
count 31204.00 31204.00   31204.00   31204.00 31204.00
mean  30429.05   410.71    7234.32   30209.69    22.23
std   15334.79  1307.14   30006.46  118199.59    81.66
min    1001.00     0.00       0.00      19.00     3.00
25%   18059.00    36.00     381.00    1819.75     4.00
50%   30096.00   106.00    1330.00    6144.00     6.00
75%   42105.00   324.00    4660.25   20522.75    14.00
max   56999.00 45246.00 1396549.00 7032755.00  4935.00

In [94]:
df.isna().sum()

State_GEOID     0
County_GEOID    0
FIPS            0
naics_2         0
naics           0
DESCRIPTION     0
emp_nf          0
emp             0
qp1_nf          0
qp1             0
ap_nf           0
ap              0
est             0
n<5             0
n5_9            0
n10_19          0
n20_49          0
n50_99          0
n100_249        0
n250_499        0
n500_999        0
n1000           0
n1000_1         0
n1000_2         0
n1000_3         0
n1000_4         0
dtype: int64

## Data Analysis

Top 10 industries by ***annual pay***

In [95]:
highest_ap = df.groupby(["naics", "DESCRIPTION"])[["ap", "qp1", "emp"]].sum()
highest_ap = highest_ap.sort_values(by="ap", ascending=False)

highest_ap.head(10)

ap       qp1      emp
naics  DESCRIPTION                                                                     
2382   Building Equipment Contractors                      149468880  33728909  2171001
2211   Electric Power Generation, Transmission and Dis...   57228297  17965188   451638
2362   Nonresidential Building Construction                 56994622  13047402   657152
2381   Foundation, Structure, and Building Exterior Co...   54423066  11455421   901696
2371   Utility System Construction                          46247985  10155569   567730
2383   Building Finishing Contractors                       45172746   9752111   811812
3330A1 3331, 3332, 3334, 3339                               37247397   9289730   488895
3345   Navigational, Measuring, Electromedical, and Co...   36803196   9441390   364204
3364   Aerospace Product and Parts Manufacturing            35180914   9310549   345099
3261   Plastics Product Manufacturing                       32199243   7618233   572426

Top 10 industries by ***pay per person***

AP:

In [96]:
highest_ap["ap_per_emp"] = highest_ap["ap"] / highest_ap["emp"]
highest_ap = highest_ap.sort_values(by="ap_per_emp", ascending=False)

highest_ap.head(10)

ap       qp1     emp  ap_per_emp
naics  DESCRIPTION                                                                               
2111   Oil and Gas Extraction                              12736937   3864511   84064      151.51
3361   Motor Vehicle Manufacturing                         10499623   2375538   79173      132.62
2211   Electric Power Generation, Transmission and Dis...  57228297  17965188  451638      126.71
3241   Petroleum and Coal Products Manufacturing            8453662   2232819   72063      117.31
2212   Natural Gas Distribution                             7548286   2252540   66339      113.78
3364   Aerospace Product and Parts Manufacturing           35180914   9310549  345099      101.94
3345   Navigational, Measuring, Electromedical, and Co...  36803196   9441390  364204      101.05
3344   Semiconductor and Other Electronic Component Ma...  21295889   5487947  214664       99.21
3250A1 3251, 3252, 3253, 3259                              23082038   5849449  236805       97.47
3311   Iron and Steel Mills and Ferroalloy Manufacturing    4242424   1080778   44071       96.26

QP1:

In [ ]:

highest_ap["qp1_per_emp"] = highest_ap["qp1"] / highest_ap["emp"]
highest_ap = highest_ap.sort_values(by="qp1_per_emp", ascending=False)

highest_ap.head(10)

Top 10 industries by ***employees per establishments***

In [97]:
highest_emp_per_est = df.groupby(["naics", "DESCRIPTION"])[["emp", "est"]].sum()
highest_emp_per_est["emp_per_est"] = highest_emp_per_est["emp"] / highest_emp_per_est["est"]
highest_emp_per_est = highest_emp_per_est.sort_values(by="emp_per_est", ascending=False)

highest_emp_per_est.head(10)

emp   est  emp_per_est
naics DESCRIPTION                                                                  
3361  Motor Vehicle Manufacturing                          79173   117       676.69
3311  Iron and Steel Mills and Ferroalloy Manufacturing    44071   152       289.94
3364  Aerospace Product and Parts Manufacturing           345099  1310       263.43
2122  Metal Ore Mining                                     29008   156       185.95
3313  Alumina and Aluminum Production and Processing       20149   147       137.07
3363  Motor Vehicle Parts Manufacturing                   464689  3700       125.59
3366  Ship and Boat Building                              110876   968       114.54
3365  Railroad Rolling Stock Manufacturing                  5785    51       113.43
3336  Engine, Turbine, and Power Transmission Equipme...   46179   430       107.39
3362  Motor Vehicle Body and Trailer Manufacturing        110704  1175        94.22